<a href="https://colab.research.google.com/github/unofficial-Jona/comp-vision/blob/main/Assignment%202/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

## load dataset

In [ ]:
df= pd.read_csv('fer2013.csv')

## preprocess dataset

In [1]:
len(df['pixels'][2])

NameError: ignored

In [ ]:
np.array(df['pixels'][0].split(' ')).reshape(1,48,48).astype('float32')/255

array([[[0.27450982, 0.3137255 , 0.32156864, ..., 0.20392157,
         0.16862746, 0.16078432],
        [0.25490198, 0.23921569, 0.22745098, ..., 0.21960784,
         0.20392157, 0.17254902],
        [0.19607843, 0.16862746, 0.21176471, ..., 0.19215687,
         0.21960784, 0.18431373],
        ...,
        [0.35686275, 0.25490198, 0.16470589, ..., 0.28235295,
         0.21960784, 0.16862746],
        [0.3019608 , 0.32156864, 0.30980393, ..., 0.4117647 ,
         0.27450982, 0.18039216],
        [0.3019608 , 0.28235295, 0.32941177, ..., 0.41568628,
         0.42745098, 0.32156864]]], dtype=float32)

In [ ]:
df['image_data']= df['pixels'].apply(lambda x: pd.Series(x.split(' ')).astype('float32').values.reshape(1,48,48))

ValueError: ignored

In [ ]:
df['image_data']

In [ ]:
df.head()

In [ ]:
df3 = df.copy()
df_melt = df3.assign(pixels = df.pixels.str.split(" "))
df_final = df_melt.pixels.apply(pd.Series).merge(df_melt,right_index=True,left_index=True)
df = df_final.drop(columns=['pixels','image_data'],axis=1)

In [ ]:
df.to_csv('image_dataset.csv')

## split data

In [ ]:
df = pd.read_csv('image_dataset.csv')

In [ ]:
set(df['Usage'])

In [ ]:
df_train = df[df['Usage']=='Training']
df_valid = df[df['Usage']=='PrivateTest']
df_test = df[df['Usage']=='PublicTest']

In [ ]:
train_input = df_train.drop(columns= ['emotion','Usage'],axis=1).values.reshape(-1,1,48,48).astype('float32')

In [ ]:
plt.imshow(train_input[0][0],cmap='gray')

train output

In [ ]:
train_output= df_train['emotion'].values
print(train_output.shape)
train_output

In [ ]:
dataset_train = TensorDataset(torch.from_numpy(train_input.astype('float32')),torch.tensor(train_output))

In [ ]:
dataset_train

In [ ]:
valid_input = df_valid.drop(columns= ['emotion','Usage'],axis=1).values.reshape(-1,1,48,48).astype('float32')
valid_output= df_valid['emotion'].values

test_input = df_test.drop(columns= ['emotion','Usage'],axis=1).values.reshape(-1,1,48,48).astype('float32')
test_output= df_test['emotion'].values


dataset_valid = TensorDataset(torch.from_numpy(valid_input.astype('float32')),torch.tensor(valid_output))
dataset_test = TensorDataset(torch.from_numpy(test_input.astype('float32')),torch.tensor(test_output))

In [ ]:
print(len(dataset_train))
print(len(dataset_valid))
print(len(dataset_test))

In [ ]:
batch_size = 256

train_loader = DataLoader(dataset_train, batch_size, shuffle=True)
val_loader = DataLoader(dataset_valid, batch_size*2)
test_loader = DataLoader(dataset_test, batch_size*2)

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
device

In [ ]:
device